# Dieser Code dient nur zum erstellen der Animation als Video
#### (Zeitabhängigkeit durch eine harmonische Schwingung $I(t) = I_0 e^{j 2\pi f t}$ als Stromquelle)

In [0]:
import sympy as sp
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation

x, y, z = sp.symbols('x y z')
r, theta, phi = sp.symbols('r theta phi')
t = sp.symbols('t')

def transform_to_xyz(expr, x, y, z):
    """
        Transformiert eine Funktion die von r, theta, phi abhängt auf eine Funkion die von x,y,z abhängt.
    """
    r, theta, phi = sp.symbols('r theta phi')
    expr = expr.subs([(r, sp.sqrt(x**2 + y**2 + z**2)),
                      (theta, sp.acos(z / sp.sqrt(x**2 + y**2 + z**2))),
                      (phi, sp.atan2(y, x))])
    return expr

def unitvector_theta(H_theta, theta, phi):
    x = H_theta * sp.cos(theta) * sp.cos(phi)
    y = H_theta * sp.cos(theta) * sp.sin(phi)
    z = H_theta * -1*sp.sin(theta)
    return x, y, z

def H_phi(theta, r, t):
    I_0 = 1
    f = 1.8e9
    c_0 = 3e8
    lamd = c_0 / f
    omega = sp.pi*2*f
    k = 2*sp.pi/lamd
    l = 0.0001
    return 1j*((k*I_0*sp.sin(theta))/(4*sp.pi*r))*sp.exp(-1j*k*r) * sp.exp(1j*omega*t)

eta = 1
E_theta = H_phi(theta, r, t)*eta
E_x,E_y,E_z =  unitvector_theta(E_theta, theta, phi)

xx = transform_to_xyz(E_x, x, y, z)
yy = transform_to_xyz(E_y, x, y, z)
zz = transform_to_xyz(E_z, x, y, z)

E_xx_func = sp.lambdify((x, y, z), xx, modules='numpy')
E_yy_func = sp.lambdify((x, y, z), yy, modules='numpy')
E_zz_func = sp.lambdify((x, y, z), zz, modules='numpy')

E_xx_func = sp.lambdify((x, y, z, t), xx, modules='numpy')
E_yy_func = sp.lambdify((x, y, z, t), yy, modules='numpy')
E_zz_func = sp.lambdify((x, y, z, t ), zz, modules='numpy')

def E(x,y,z,t):
    return np.array([np.real(E_xx_func(x,y,z,t)),
                     np.real(E_yy_func(x,y,z,t)),
                     np.real(E_zz_func(x,y,z,t))])

start = -1
stop = 1
samples = 2 ** 6

x_range = np.linspace(start, stop, samples)
y_range = np.linspace(start, stop, samples)
z_range = np.linspace(start, stop, samples)
t_range = np.linspace(0, 2*np.pi, samples)
X, Y, Z, T = np.meshgrid(x_range, y_range, z_range, t_range, indexing='ij')
E_field = np.vectorize(E, signature='(),(),(),()->(n)')(X, Y, Z, T)

x = 0
y = 1
z = 2

plt.imshow(np.abs(E_field[30,:,:,1,x]))
plt.colorbar()

### Indizierung des 5 Dimensionalen Feldes E_field

E Feld ist hat jetzt 5 Dimensionen, die Indizierung ist wie folgt definiert:

- E_field[t,z,y,x,Komponente]

In [0]:
E_field.shape

### Zum Video erstellen

In [0]:
x = 0
y = 1
z = 2

fig = plt.figure()

def animate(i):
    plt.clf()
    plt.imshow(np.abs(E_field[i, :, :, 20, x]))
    plt.title(r'X-Komponente von $\vec E(\vec r, t)$'+'\n'+ r'mit der Stromquelle $I(t) = I_0 e^{1j\omega t})$'+'\n(Schnitt der zy-Ebene)')
    plt.xlabel('y')
    plt.ylabel('z')
    plt.colorbar()

ani = animation.FuncAnimation(fig, animate, frames=len(t_range), interval=200)
ani.save('animation.mp4', writer='ffmpeg')